1. The first machine learning improvement I decided to do was to improve the ensemble accuracy. Firstly I created an ensemble for the imdb dataset and calculated its accuracy.

In [2]:
 pip install scikit-learn

In [3]:
 pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset

imdb_dataset = load_dataset("imdb")['train']

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
train_data = []
train_data_labels = []
for item in imdb_dataset:
  train_data.append(item['text'])
  train_data_labels.append(item['label'])
print(train_data[-1])
print(train_data_labels[-1])

The story centers around Barry McKenzie who must go to England if he wishes to claim his inheritance. Being about the grossest Aussie shearer ever to set foot outside this great Nation of ours there is something of a culture clash and much fun and games ensue. The songs of Barry McKenzie(Barry Crocker) are highlights.
1


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer='word',max_features=1000,lowercase=True,stop_words='english',ngram_range=(1,2))
features = vectorizer.fit_transform(train_data).toarray()

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(features,train_data_labels,train_size=0.75,random_state=123)

In [8]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
model_nb = MultinomialNB()
model_dt = DecisionTreeClassifier()
model_rf = RandomForestClassifier()

In [9]:
model_nb = model_nb.fit(X=X_train,y=y_train)
model_dt = model_dt.fit(X=X_train,y=y_train)
model_rf = model_rf.fit(X=X_train,y=y_train)

In [10]:
y_pred_nb = model_nb.predict(X_val)
y_pred_dt = model_dt.predict(X_val)
y_pred_rf = model_rf.predict(X_val)

In [11]:
from sklearn.ensemble import VotingClassifier
ensemble = VotingClassifier(estimators=[('nb', model_nb), ('dt', model_dt), ('rf', model_rf)], voting='hard')
ensemble.fit(X_train, y_train)
y_pred_ensemble = ensemble.predict(X_val)

In [12]:
from sklearn.metrics import accuracy_score, confusion_matrix
print("Naive Bayes Accuracy:", accuracy_score(y_val, y_pred_nb))
print("Naive Bayes Confusion Matrix:\n", confusion_matrix(y_val, y_pred_nb))
print()

print("Decision Tree Accuracy:", accuracy_score(y_val, y_pred_dt))
print("Decision Tree Confusion Matrix:\n", confusion_matrix(y_val, y_pred_dt))
print()

print("Random Forest Accuracy:", accuracy_score(y_val, y_pred_rf))
print("Random Forest Confusion Matrix:\n", confusion_matrix(y_val, y_pred_rf))
print()

print("Ensemble Accuracy:", accuracy_score(y_val, y_pred_ensemble))
print("Ensemble Confusion Matrix:\n", confusion_matrix(y_val, y_pred_ensemble))

Naive Bayes Accuracy: 0.81728
Naive Bayes Confusion Matrix:
 [[2463  622]
 [ 520 2645]]

Decision Tree Accuracy: 0.7016
Decision Tree Confusion Matrix:
 [[2193  892]
 [ 973 2192]]

Random Forest Accuracy: 0.812
Random Forest Confusion Matrix:
 [[2521  564]
 [ 611 2554]]

Ensemble Accuracy: 0.82096
Ensemble Confusion Matrix:
 [[2516  569]
 [ 550 2615]]


To improve the ensemble accuracy I experimented with adding weights to each diiferent model to see if it would improve the overall accuracy. I found that adding higher weights to the models with the higher indivudual accuracy scores(Naive Bayes and Random Forest), gave a slightly higher accuracy score to the ensemble.

In [13]:
from sklearn.ensemble import VotingClassifier
ensemble = VotingClassifier(estimators=[('nb', model_nb), ('dt', model_dt), ('rf', model_rf)], voting='hard', weights=[2, 1, 2])
ensemble.fit(X_train, y_train)
y_pred_ensemble = ensemble.predict(X_val)

In [14]:
from sklearn.metrics import accuracy_score, confusion_matrix
print("Naive Bayes Accuracy:", accuracy_score(y_val, y_pred_nb))
print("Naive Bayes Confusion Matrix:\n", confusion_matrix(y_val, y_pred_nb))
print()

print("Decision Tree Accuracy:", accuracy_score(y_val, y_pred_dt))
print("Decision Tree Confusion Matrix:\n", confusion_matrix(y_val, y_pred_dt))
print()

print("Random Forest Accuracy:", accuracy_score(y_val, y_pred_rf))
print("Random Forest Confusion Matrix:\n", confusion_matrix(y_val, y_pred_rf))
print()

print("Ensemble Accuracy:", accuracy_score(y_val, y_pred_ensemble))
print("Ensemble Confusion Matrix:\n", confusion_matrix(y_val, y_pred_ensemble))

Naive Bayes Accuracy: 0.81728
Naive Bayes Confusion Matrix:
 [[2463  622]
 [ 520 2645]]

Decision Tree Accuracy: 0.7016
Decision Tree Confusion Matrix:
 [[2193  892]
 [ 973 2192]]

Random Forest Accuracy: 0.812
Random Forest Confusion Matrix:
 [[2521  564]
 [ 611 2554]]

Ensemble Accuracy: 0.8216
Ensemble Confusion Matrix:
 [[2520  565]
 [ 550 2615]]


2. For the second idea I experimented with sentiment lexicons to improve the accuracy of the decision tree classifier. First I looked at the original accuracy for the decision tree classifier which was 0.7016 and then used the VADER lexicon to improve its accuracy.

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

tfidf_vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2))
X = tfidf_vectorizer.fit_transform(train_data)

X_train, X_test, y_train, y_test = train_test_split(X, train_data_labels, test_size=0.2, random_state=42)

dt_classifier = DecisionTreeClassifier()
dt_classifier.fit(X_train, y_train)

dt_predictions = dt_classifier.predict(X_test)

dt_accuracy = accuracy_score(y_test, dt_predictions)

print(f"Decision Tree Accuracy with TF-IDF: {dt_accuracy}")



Decision Tree Accuracy with TF-IDF: 0.7158


In [16]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

df = pd.DataFrame(imdb_dataset)

nltk.download('vader_lexicon')

vader_analyzer = SentimentIntensityAnalyzer()

df['compound_score'] = df['text'].apply(lambda x: vader_analyzer.polarity_scores(x)['compound'])

X_train, X_val, y_train, y_val = train_test_split(df[['compound_score']], df['label'], test_size=0.2, random_state=42)

dt_model_vader = DecisionTreeClassifier(random_state=42)
dt_model_vader.fit(X_train, y_train)

y_pred_vader = dt_model_vader.predict(X_val)

accuracy_vader = accuracy_score(y_val, y_pred_vader)
print(f"Accuracy with VADER Compound Scores: {accuracy_vader}")


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


Accuracy with VADER Compound Scores: 0.6696


While the use of the vader an tf-idf didn't improve the accuracy overall, I noticed that with just the use of tf-idf it improved its accuracy.

In [17]:
pip install gradio

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_tfidf, y_train)

y_pred = naive_bayes.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.8514
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.85      0.85      2515
           1       0.85      0.85      0.85      2485

    accuracy                           0.85      5000
   macro avg       0.85      0.85      0.85      5000
weighted avg       0.85      0.85      0.85      5000



In [19]:
import gradio as gr

def predict_review(review):
    review_tfidf = tfidf_vectorizer.transform([review])
    prediction = naive_bayes.predict(review_tfidf)[0]
    return prediction

iface = gr.Interface(
    fn=predict_review,
    inputs="text",
    outputs="text",
    live=True,
    title="Film Review Classifier",
    description="Enter a film review to get sentiment prediction.",
    theme="compact"
)

iface.launch()


/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:528: UserWarning: Cannot load compact. Caught Exception: The space compact does not exist
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://bb9930cec2a7358983.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
